In [1]:
import requests                 # To download the html data from a site
from bs4 import BeautifulSoup   # To parse the html data
import time                     # To add a delay between each requests
import pandas as pd             # To store our data in a DataFrame

In [2]:
url = "https://trace.tennessee.edu/utk_graddiss/index.html"

In [3]:
r = requests.get(url)

In [4]:
r


<Response [200]>

In [5]:
print(r.text[:200])


<!DOCTYPE html>
<html lang="en">
<head><!-- inj yui3-seed: --><script type='text/javascript' src='//cdnjs.cloudflare.com/ajax/libs/yui/3.6.0/yui/yui-min.js'></script><script type='text/javascript' sr


In [6]:
mainpage = BeautifulSoup(r.text, "html.parser")

In [7]:
#prettify to make things easy for the beautiful soup can work with
print(mainpage.prettify()[:200])

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- inj yui3-seed: -->
  <script src="//cdnjs.cloudflare.com/ajax/libs/yui/3.6.0/yui/yui-min.js" type="text/javascript">
  </script>
  <script src="//ajax.g


In [8]:
mainpage.title

<title>
Doctoral Dissertations | Graduate School | University of Tennessee, Knoxville
</title>

In [10]:
r.text.title

<function str.title()>

In [11]:
mainpage.a

<a data-scroll="" href="https://trace.tennessee.edu" title="Home">Home</a>

In [12]:
mainpage.find("a")

<a data-scroll="" href="https://trace.tennessee.edu" title="Home">Home</a>

In [13]:
mainpage.find_all("a")

[<a data-scroll="" href="https://trace.tennessee.edu" title="Home">Home</a>,
 <a data-scroll="" href="https://trace.tennessee.edu/do/search/advanced/" title="Search"><i class="icon-search"></i> Search</a>,
 <a data-scroll="" href="https://trace.tennessee.edu/communities.html" title="Browse">Browse Collections</a>,
 <a data-scroll="" href="/cgi/myaccount.cgi?context=utk_graddiss" title="My Account">My Account</a>,
 <a data-scroll="" href="https://trace.tennessee.edu/about.html" title="About">About</a>,
 <a data-scroll="" href="https://network.bepress.com" title="Digital Commons Network"><img alt="DC Network" height="16" src="/assets/md5images/8e240588cf8cd3a028768d4294acd7d3.png" style="vertical-align:top;" width="16"/> Digital Commons Network™</a>,
 <a accesskey="2" class="skiplink" href="#main">Skip to main content</a>,
 <a href="https://trace.tennessee.edu" id="banner_link" title="TRACE: Tennessee Research and Creative Exchange">
 <img alt="TRACE: Tennessee Research and Creative Exch

In [14]:
mainpage.select(".article-listing a")

[<a href="https://trace.tennessee.edu/utk_graddiss/8681">Stability of Quantum Computers</a>,
 <a href="https://trace.tennessee.edu/utk_graddiss/8076">Understanding host-microbe interactions in maize kernel and sweetpotato leaf metagenomic profiles.</a>,
 <a href="https://trace.tennessee.edu/utk_graddiss/8080">FABRICATION, MEASUREMENTS, AND MODELING OF SEMICONDUCTOR RADIATION DETECTORS FOR IMAGING AND DETECTOR RESPONSE FUNCTIONS</a>,
 <a href="https://trace.tennessee.edu/utk_graddiss/8086">Development and Characterization of Stable Low-cost Salt Hydrate-based Phase Change Materials for Thermal Energy Storage Applications</a>,
 <a href="https://trace.tennessee.edu/utk_graddiss/8078">The Light from Pisgah</a>,
 <a href="https://trace.tennessee.edu/utk_graddiss/8584">Clickable Lipid Precursors as Chemical Tools for Imaging and Tracking Lipids</a>,
 <a href="https://trace.tennessee.edu/utk_graddiss/8703">Retinoic Acid, Insulin, and Cycloheximide Alter  Glycogen Homeostasis in Differentiatin

In [16]:
mainpage.select(".article-listing a")[0]

<a href="https://trace.tennessee.edu/utk_graddiss/8681">Stability of Quantum Computers</a>

In [17]:
mainpage.select(".article-listing a")[0].get("href")

'https://trace.tennessee.edu/utk_graddiss/8681'

In [19]:
url_test=mainpage.select(".article-listing a")[0].get("href")

In [20]:
r_test=requests.get(url_test)

In [22]:
dissertpage_test=BeautifulSoup(r_test.content, "html.parser")

In [25]:
dissertpage_test.select("#publication_date p")

[<p>5-2024</p>]

In [26]:
dissertpage_test.select("#publication_date p")[0].text

'5-2024'

In [27]:
dissertlinks = mainpage.select(".article-listing a")

In [28]:
urls = []  # We create an empty list before filling it during the loop

for link in dissertlinks:
    urls.append(link.get("href"))

In [29]:
urls[:5]

['https://trace.tennessee.edu/utk_graddiss/8681',
 'https://trace.tennessee.edu/utk_graddiss/8076',
 'https://trace.tennessee.edu/utk_graddiss/8080',
 'https://trace.tennessee.edu/utk_graddiss/8086',
 'https://trace.tennessee.edu/utk_graddiss/8078']

In [30]:
ls = []                                            # Create an empty list

for url in urls:                                   # For each element of our list of sites
    r = requests.get(url)                                     # Send a request to the site
    dissertpage = BeautifulSoup(r.text, "html.parser")        # Parse the result
    date = dissertpage.select("#publication_date p")[0].text  # Get the date
    major = dissertpage.select("#department p")[0].text       # Get the major
    advisor = dissertpage.select("#advisor1 p")[0].text            # Get the advisor
    ls.append((date, major, advisor))                              # Store the results in the list
    time.sleep(0.1)                                           # Add a delay at each iteration

In [31]:
cols = ["Date", "Major", "Advisor"]

In [32]:
df = pd.DataFrame(ls, columns=cols)

In [33]:
df

,Date,Major,Advisor
0,5-2024,Data Science and Engineering,Travis Humble
1,5-2023,Life Sciences,Bode A. Olukolu
2,5-2023,Nuclear Engineering,Erik Lukosi
3,5-2023,Energy Science and Engineering,Kyle R. Gluesenkamp
4,5-2023,English,Margaret Lazarus Dean
...,...,...,...
95,8-2023,Psychology,Todd M. Moore
96,8-2023,Animal Science,Brynn H. Voy
97,8-2023,Child and Family Studies,Megan L. Haselschwerdt
98,5-2023,Business Administration,Eric Kelley


In [34]:
df.to_csv('dissertations_data.csv', index=False)

In [36]:
%pwd

'c:\\Users\\dell\\ML project\\Log_model'